
# Capítulo 3 — EDA e Pré-processamento de Dados

**Curso:** CECIERJ – IA e ML para Soluções Práticas  
**Objetivo:** realizar **Análise Exploratória de Dados (EDA)** e aplicar técnicas de **pré-processamento** fundamentais antes de treinar modelos.

---
## Passos abordados
1. Carregamento de dataset (`Wine` do scikit-learn).  
2. EDA: dimensões, estatísticas, distribuições, correlações.  
3. Tratamento de valores ausentes.  
4. Normalização/Padronização.  
5. *Encoding* de variáveis categóricas.  
6. *Train/Test Split* e boas práticas contra *data leakage*.  
7. *Pipeline* com `ColumnTransformer`.


In [ ]:

import pandas as pd
from sklearn.datasets import load_wine

ds = load_wine(as_frame=True)
df = ds.frame.copy()
print("Shape:", df.shape)
df.head()


In [ ]:

# Estatísticas descritivas
display(df.describe())

# Tipos de dados e checagem de ausentes
print(df.dtypes.head())
print("Total de ausentes:", df.isna().sum().sum())


In [ ]:

import matplotlib.pyplot as plt

df.hist(figsize=(12,8), bins=20)
plt.suptitle("Distribuições das variáveis (Wine)", y=1.02)
plt.show()


In [ ]:

import seaborn as sns

plt.figure(figsize=(10,8))
sns.heatmap(df.corr(numeric_only=True), cmap="coolwarm", center=0)
plt.title("Matriz de correlação (Wine)")
plt.show()


In [ ]:

# Introduzindo valores ausentes artificiais em 1% das linhas
import numpy as np

df2 = df.copy()
rng = np.random.default_rng(42)
ix = rng.choice(df2.index, size=int(len(df2)*0.01), replace=False)
df2.loc[ix, "alcohol"] = None
print("Ausentes em 'alcohol':", df2["alcohol"].isna().sum())

# Tratamento: preenchendo com mediana
df2["alcohol"].fillna(df2["alcohol"].median(), inplace=True)
print("Ausentes após fillna:", df2["alcohol"].isna().sum())


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["target"])
y = df["target"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Treino:", X_train_scaled.shape, "Teste:", X_test_scaled.shape)


In [ ]:

# Exemplo didático com variáveis categóricas simuladas
toy = pd.DataFrame({
    "bairro": ["Centro","Norte","Centro","Sul","Sul","Norte","Sul","Centro"],
    "quartos": [2,3,2,4,3,2,4,1],
    "area": [60, 80, 55, 120, 90, 50, 110, 40],
    "preco": [320, 480, 300, 800, 520, 260, 750, 220]
})
toy


In [ ]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

Xc = toy[["bairro", "quartos", "area"]]
yc = toy["preco"]

num_cols = ["quartos","area"]
cat_cols = ["bairro"]

ct = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(drop="first"), cat_cols),
])
Xc_transformed = ct.fit_transform(Xc)
print("Shape após ColumnTransformer:", Xc_transformed.shape)


In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = Pipeline([("prep", ct), ("lr", LinearRegression())])
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.25, random_state=42)
reg.fit(Xc_train, yc_train)
pred = reg.predict(Xc_test)
print("MSE:", round(mean_squared_error(yc_test, pred),2))



---
## Boas práticas de Pré-processamento
- **Nunca aplique `fit` em dados de teste** → evite *data leakage*.  
- Use **`Pipeline`/`ColumnTransformer`** para manter consistência.  
- Sempre cheque **estatísticas e distribuições** antes de normalizar.  
- Documente as transformações e salve objetos (`scaler`, `encoder`).  
- Pense em escalabilidade (colunas novas, categorias raras).
